In [1]:
from torch_geometric.datasets import Planetoid

In [2]:
# Load the dataset
dataset = Planetoid("./", "Cora")
data = dataset[0]
print("Cora", data)

Cora Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [3]:
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [16]:
# We will implement SGCN from scrath using just torch

In [17]:
# Adjacency matrix
A = torch.eye(data.num_nodes) + torch.sparse.FloatTensor(data.edge_index, torch.ones(data.edge_index.shape[1]))
A.shape 

torch.Size([2708, 2708])

In [23]:
# According to the original paper we are implementing the S
degree_hat = A.sum(dim=1).pow(-0.5)
D_inv_sqrt = torch.diag(degree_hat)
S = D_inv_sqrt.mm(A).mm(D_inv_sqrt)

In [24]:
# Graph propagation
def graph_propagate(x, S, K): 
    # x --> features vector
    # s --> simple conv operation
    # K hops or number of times we want to do the operation
    for _ in range(K):
        x = torch.mm(S, x)
    return x

In [35]:
# Model implementationl
import torch.nn as nn
class SGC(nn.Module):

    def __init__(self, n_features, n_classes):
        super(SGC, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(n_features, n_classes))
        self.reset_parameters()
    
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, x):
        return torch.mm(x, self.weight)

In [36]:
# Performing simple conv operation before passing to main model
K = 2  
preprocessed_features = graph_propagate(data.x, S, K)

In [37]:
import torch.optim as optim
model = SGC(preprocessed_features.size(1), dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [38]:
model.train()

num_epochs = 200
for epoch in range(num_epochs):
    optimizer.zero_grad()
    out = model(preprocessed_features)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


/Users/arindams_mac_m2_pro/opt/anaconda3/envs/py310/lib/python3.10/site-packages/torch/optim/optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Epoch 0, Loss: 1.9550
Epoch 10, Loss: 0.9269
Epoch 20, Loss: 0.4479
Epoch 30, Loss: 0.2556
Epoch 40, Loss: 0.1748
Epoch 50, Loss: 0.1389
Epoch 60, Loss: 0.1218
Epoch 70, Loss: 0.1129
Epoch 80, Loss: 0.1072
Epoch 90, Loss: 0.1029
Epoch 100, Loss: 0.0991
Epoch 110, Loss: 0.0957
Epoch 120, Loss: 0.0925
Epoch 130, Loss: 0.0897
Epoch 140, Loss: 0.0871
Epoch 150, Loss: 0.0849
Epoch 160, Loss: 0.0829
Epoch 170, Loss: 0.0811
Epoch 180, Loss: 0.0795
Epoch 190, Loss: 0.0781


In [40]:
# Model performance
model.eval()
_, pred = out.max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8000


In [42]:
out.shape

torch.Size([2708, 7])

In [43]:
## Will do the above using torch_geometric now

In [44]:
from torch_geometric.nn import SGConv

In [53]:
#SGC net 
class SGCNet(nn.Module):

    def __init__(self, n_features, n_classes, K):
        super(SGCNet, self).__init__()
        self.conv = SGConv(n_features, 
                           n_classes, 
                           K=K, 
                           cached=True)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv(x, edge_index)
        return x



In [54]:
# Initialize and train the model
model_tg = SGCNet(dataset.num_features, dataset.num_classes, K=2)
optimizer_tg = optim.Adam(model_tg.parameters(), lr=0.01, weight_decay=5e-4)
criterion_tg = torch.nn.CrossEntropyLoss()


In [55]:
model_tg.train()
for epoch in range(200):
    optimizer_tg.zero_grad()
    out_tg = model_tg(data)
    loss = criterion_tg(out_tg[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_tg.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 1.9431
Epoch 10, Loss: 0.9182
Epoch 20, Loss: 0.4431
Epoch 30, Loss: 0.2523
Epoch 40, Loss: 0.1721
Epoch 50, Loss: 0.1366
Epoch 60, Loss: 0.1197
Epoch 70, Loss: 0.1109
Epoch 80, Loss: 0.1054
Epoch 90, Loss: 0.1012
Epoch 100, Loss: 0.0975
Epoch 110, Loss: 0.0941
Epoch 120, Loss: 0.0911
Epoch 130, Loss: 0.0883
Epoch 140, Loss: 0.0858
Epoch 150, Loss: 0.0836
Epoch 160, Loss: 0.0816
Epoch 170, Loss: 0.0799
Epoch 180, Loss: 0.0783
Epoch 190, Loss: 0.0769


In [56]:
# Model performace
model_tg.eval()
_, pred = out_tg.max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.7990
